# Human Activity Recognition - SSL JOINT

Last Update : 24 July 2019

In [1]:
N_THREADS = 8
# Nota Bene : notebooks don't deallocate GPU memory
IS_FORCE_CPU = False # can also be set in the trainer

## Environment

In [2]:
cd ..

/master


In [3]:
%autosave 600
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# CENTER PLOTS
from IPython.core.display import HTML
display(HTML(""" <style> .output_png {display: table-cell; text-align: center; margin:auto; }
.prompt display:none;}  </style>"""))

import os
if IS_FORCE_CPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = ""

import sys
sys.path.append("notebooks")

import numpy as np
import matplotlib.pyplot as plt
from functools import partial
import pandas as pd
import h5py


import torch
torch.set_num_threads(N_THREADS)

Autosaving every 600 seconds


# Dataset

In [4]:
from skssl.transformers.neuralproc.datasplit import CntxtTrgtGetter, GetRandomIndcs, get_all_indcs
from utils.data.tsdata import get_timeseries_dataset, SparseMultiTimeSeriesDataset

get_cntxt_trgt_test = CntxtTrgtGetter(contexts_getter=GetRandomIndcs(min_n_indcs=0.1, max_n_indcs=0.5),
                                     targets_getter=get_all_indcs,
                                     is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

get_cntxt_trgt = CntxtTrgtGetter(contexts_getter=GetRandomIndcs(min_n_indcs=0.01, max_n_indcs=0.5),
                                 targets_getter=GetRandomIndcs(min_n_indcs=0.5, max_n_indcs=0.99),
                                 is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

In [5]:
data_both = get_timeseries_dataset("har")(split="both")

def cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=False):
    def mycollate(batch):
        min_length = min([v.size(0) for b in batch for k,v in b[0].items() if "X" in k])
        # chose first min_legth of each (assumes that randomized)
        
        batch = [({k:v[:min_length, ...] for k,v in b[0].items()}, b[1]) for b in batch]        
        collated = torch.utils.data.dataloader.default_collate(batch)
        
        X = collated[0]["X"]
        y = collated[0]["y"]
        
        if is_repeat_batch:
            
            X = torch.cat([X,X], dim=0)
            y = torch.cat([y,y], dim=0)
            collated[1] = torch.cat([collated[1], collated[1]], dim=0) # targets
        
        collated[0]["X"], collated[0]["y"], collated[0]["X_trgt"], collated[0]["y_trgt"] = get_cntxt_trgt(X, y)
        
        return collated
    return mycollate

In [6]:
X_DIM = 1  # 1D spatial input (although actually 2 but the first is for sparse channels)
Y_DIM = data_both.data.shape[-1] # multiple channels
N_TARGETS = len(np.unique(data_both.targets))

sampling_percentages = [0.5]
label_percentages = [N_TARGETS, N_TARGETS*2, 0.01, 0.05, 0.1, 0.3, 0.5, 1]

## Model

In [7]:
import torch.nn as nn
from skssl.transformers import GlobalNeuralProcess, NeuralProcessLoss, AttentiveNeuralProcess
from skssl.utils.helpers import rescale_range
from skssl.predefined import UnetCNN, CNN, MLP, SparseSetConv, SetConv, MlpRBF, GaussianRBF, BatchSparseSetConv
from skssl.transformers.neuralproc.datasplit import precomputed_cntxt_trgt_split
from utils.helpers import count_parameters

In [8]:
from copy import deepcopy

models = {}

unet = partial(UnetCNN,
               Conv=torch.nn.Conv1d,
               Pool=torch.nn.MaxPool1d,
               upsample_mode="linear",
               n_layers=18,
               is_double_conv=True,
               is_depth_separable=True,
               Normalization=torch.nn.BatchNorm1d,
               is_chan_last=True,
               bottleneck=None,
               kernel_size=7,
               max_nchannels=256,
              is_force_same_bottleneck=True,
               _is_summary=True,
              )

kwargs = dict(x_dim=X_DIM, 
              y_dim=Y_DIM,
              min_std=5e-3,
                n_tmp_queries=128,
                r_dim=64,
              keys_to_tmp_attn=partial(SetConv, RadialBasisFunc=GaussianRBF),
              TmpSelfAttn=unet,
              tmp_to_queries_attn=partial(SetConv, RadialBasisFunc=GaussianRBF),
              is_skip_tmp=False,
              is_use_x=False,
              get_cntxt_trgt=precomputed_cntxt_trgt_split,
              is_encode_xy=False,
             Classifier=partial(MLP, input_size=256, output_size=N_TARGETS, dropout=0.5, hidden_size=64, n_hidden_layers=1))

models["ssl_classifier_gnp_large_shared_bottleneck_sup"] = partial(GlobalNeuralProcess, **kwargs)

kwargs_bis = deepcopy(kwargs)
kwargs_bis["TmpSelfAttn"] = partial(unet, n_layers=14, max_nchannels=128)
kwargs_bis["Classifier"] = partial(MLP, input_size=128, output_size=N_TARGETS, dropout=0.5, hidden_size=64, n_hidden_layers=3)
kwargs_bis["r_dim"] = 16

models["ssl_classifier_gnp_small_shared_bottleneck_sup"] = partial(GlobalNeuralProcess, **kwargs_bis)

kwargs_bis = deepcopy(kwargs)
kwargs_bis["TmpSelfAttn"] = partial(unet, n_layers=10, bottleneck=True, max_nchannels=128)
kwargs_bis["Classifier"] = partial(MLP, input_size=64, output_size=N_TARGETS,  dropout=0.5, hidden_size=64, n_hidden_layers=3)
kwargs_bis["r_dim"] = 16

models["ssl_classifier_gnp_mini_shared_bottleneck_sup"] = partial(GlobalNeuralProcess, **kwargs_bis)

In [9]:
from utils.helpers import count_parameters
for k,v in models.items():
    print(k, "- N Param:", count_parameters(v()))

ssl_classifier_gnp_large_shared_bottleneck_sup - N Param: 1023774
ssl_classifier_gnp_small_shared_bottleneck_sup - N Param: 105654
ssl_classifier_gnp_mini_shared_bottleneck_sup - N Param: 38262


/master/skssl/predefined/mlp.py:61: UserWarning: hidden_size=32 larger than output=16 and input=9. Setting it to 16.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/master/skssl/predefined/mlp.py:61: UserWarning: hidden_size=32 larger than output=18 and input=16. Setting it to 18.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))


# Training

In [10]:
from ntbks_helpers import train_models_
from skorch.dataset import CVSplit
from utils.data.ssldata import get_train_dev_test_ssl

In [11]:
N_EPOCHS = 100 
BATCH_SIZE = 32
IS_RETRAIN = True # if false load precomputed
chckpnt_dirname="results/challenge/har/"

In [12]:
from skssl.utils.helpers import HyperparameterInterpolator

n_steps_per_epoch = len(data_both)//BATCH_SIZE
hi=HyperparameterInterpolator(1e-5, 10, N_EPOCHS*n_steps_per_epoch, start_step=n_steps_per_epoch*10, mode="linear")

In [16]:
data_trainers = {}

for sampling_perc in sampling_percentages:
    for label_perc in [1]:#label_percentages:
        data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                          n_labels=label_perc, 
                                                          data_perc=sampling_perc, 
                                                          dev_size=0)
        
        # add test as unlabeled data
        data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
        data_train.targets = np.concatenate([data_train.targets, np.ones_like(data_test.targets)], axis=0)
        data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)
        
        data_trainers.update(train_models_({"{}%har_{}%lab".format(int(sampling_perc*100), 
                                                                   int(label_perc*100)): (data_train, data_test)}, 
                              models, 
                              criterion=partial(NeuralProcessLoss, 
                                                ssl_loss="supervised",
                                                get_lambda_sup=lambda: hi(True)),
                                patience=15,
                              chckpnt_dirname=chckpnt_dirname,
                              max_epochs=N_EPOCHS,
                              batch_size=BATCH_SIZE,
                              is_retrain=IS_RETRAIN,
                              #train_split=CVSplit(0.05),
                              iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                              iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_test)))


--- Training 50%har_100%lab/ssl_classifier_gnp_large_shared_bottleneck_sup ---



  epoch    train_loss    valid_acc    valid_loss    cp      dur
-------  ------------  -----------  ------------  ----  -------
      1        7.5321       0.4028        1.3262     +  19.6591


      2        3.3597       0.4255        1.2522     +  19.6911


      3        2.5699       0.4744        1.1186     +  19.4092


      4        2.1422       0.3424        1.2435        19.2636


...

     22        8.2308       0.8619        0.5763        10.2670


     23        8.1264       0.8426        0.7242        10.2126


     24        8.0781       0.8687        0.6566        10.2173


     25        8.2907       0.8724        0.6768        10.2268


Stopping since valid_loss has not improved in the last 15 epochs.
Re-initializing module.
Re-initializing optimizer.
50%har_100%lab/ssl_classifier_gnp_mini_shared_bottleneck_sup best epoch: 11 val_loss: 0.563553226977071


In [17]:
for k,t in data_trainers.items():
    print()
    for e, h in enumerate(t.history[::-1]):
        if h["valid_loss_best"]:
            print(k, "epoch:", len(t.history)-e, 
                  "val_loss:", h["valid_loss"], 
                  "val_acc:", h["valid_acc"])
            break


50%har_100%lab/ssl_classifier_gnp_large_shared_bottleneck_sup epoch: 22 val_loss: 0.5084086644030407 val_acc: 0.8557855446216491

50%har_100%lab/ssl_classifier_gnp_small_shared_bottleneck_sup epoch: 25 val_loss: 0.4774247264311926 val_acc: 0.9124533423820834

50%har_100%lab/ssl_classifier_gnp_mini_shared_bottleneck_sup epoch: 11 val_loss: 0.563553226977071 val_acc: 0.8876823888700374


In [42]:
for i in range(100*n_steps_per_epoch):
    x = hi(True)
    if i % n_steps_per_epoch == 0:
        print(i//n_steps_per_epoch, x)

0 1.0004304823679598e-05
1 1.1486478813866367e-05
2 1.3188242248387777e-05
3 1.514212809866449e-05
4 1.7385489213651198e-05
5 1.9961212402148445e-05
6 2.291853830899504e-05
7 2.6314002758887478e-05
8 3.0212517563695364e-05
9 3.4688611455294726e-05
10 3.9827854867094556e-05
11 4.5728495802106476e-05
12 5.2503338060793106e-05
13 6.0281897735174225e-05
14 6.921287919534444e-05
15 7.946701790236032e-05
16 9.124034439415184e-05
17 0.00010475793184276736
18 0.00012027819882579596
19 0.00013809784956895856
20 0.0001585575461035302
21 0.0001820484207744256
22 0.00020901955359994335
23 0.00023998655742943475
24 0.00027554143502317284
25 0.0003163638964943012
26 0.0003632343534707939
27 0.00041704883838956735
28 0.00047883613413804584
29 0.00054977744151416
30 0.0006312289604917681
31 0.0007247478169823302
32 0.0008321218307402155
33 0.0009554036934909864
34 0.0010969502106731646
35 0.0012594673569861518
36 0.0014460620070807454
37 0.001660301330338795
38 0.0019062809852045635
39 0.0021887034167